# Estudo prático da estrutura das redes neurais artificiais - Classificação binária

#### Sabrina Otoni da Silva - 2023

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score

import tensorflow as tf
from tensorflow.keras import backend as k
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Dense, Dropout

from scikeras.wrappers import KerasClassifier, KerasRegressor

In [2]:
previsores = pd.read_csv('entradas_breast.csv')
classe = pd.read_csv('saidas_breast.csv')

previsores_treinamento, previsores_teste, classe_treinamento, classe_teste = train_test_split(previsores, classe, test_size=0.25)

classificador = Sequential([
               tf.keras.layers.Dense(units=16, activation = 'relu', kernel_initializer = 'random_uniform', input_dim=30),
               tf.keras.layers.Dense(units=16, activation = 'relu', kernel_initializer = 'random_uniform'),
               tf.keras.layers.Dense(units=1, activation = 'sigmoid')])

In [3]:
otimizador = tf.keras.optimizers.Adam(learning_rate = 0.001, weight_decay = 0.0001, clipvalue = 0.5)

classificador.compile(optimizer = otimizador, loss = 'binary_crossentropy',
                      metrics = ['binary_accuracy'])

classificador.fit(previsores_treinamento, classe_treinamento,
                  batch_size = 10, epochs = 100)

Epoch 1/100
43/43 [==============================] - 1s 2ms/step - loss: 1.0349 - binary_accuracy: 0.5516
Epoch 2/100
43/43 [==============================] - 0s 2ms/step - loss: 0.4675 - binary_accuracy: 0.7324
Epoch 3/100
43/43 [==============================] - 0s 2ms/step - loss: 0.3994 - binary_accuracy: 0.8122
Epoch 4/100
43/43 [==============================] - 0s 2ms/step - loss: 0.4492 - binary_accuracy: 0.7958
Epoch 5/100
43/43 [==============================] - 0s 2ms/step - loss: 0.3661 - binary_accuracy: 0.8404
Epoch 6/100
43/43 [==============================] - 0s 2ms/step - loss: 0.4135 - binary_accuracy: 0.8404
Epoch 7/100
43/43 [==============================] - 0s 2ms/step - loss: 0.3605 - binary_accuracy: 0.8568
Epoch 8/100
43/43 [==============================] - 0s 2ms/step - loss: 0.3548 - binary_accuracy: 0.8803
Epoch 9/100
43/43 [==============================] - 0s 2ms/step - loss: 0.5280 - binary_accuracy: 0.7934
Epoch 10/100
43/43 [==========================

In [4]:
pesos0 = classificador.layers[0].get_weights()
print(pesos0)
print(len(pesos0))

[array([[-1.04531214e-01, -1.10915914e-01, -1.63163587e-01,
         6.10635802e-02, -1.22968420e-01,  1.31991073e-01,
         6.59017339e-02, -4.84190211e-02,  1.20357163e-01,
        -2.59578861e-02,  1.44198045e-01,  1.46997079e-01,
         7.55535737e-02,  3.93782407e-02, -1.29439225e-02,
        -5.48320785e-02],
       [-9.30342972e-02, -1.98527217e-01, -5.42400219e-02,
        -1.74055044e-02, -3.08072448e-01,  2.37119012e-02,
         5.67957992e-03, -3.25319916e-02,  4.32547778e-02,
        -1.01894345e-02,  2.72874951e-01, -2.94073652e-02,
        -1.24002032e-01,  2.10220948e-01,  1.36659175e-01,
        -2.18727529e-01],
       [ 1.96520150e-01, -6.36120215e-02, -1.51972607e-01,
        -3.48054357e-02, -3.56756174e-03, -4.51598056e-02,
        -9.15048346e-02,  3.10902949e-02, -1.01244055e-01,
         9.17708278e-02,  6.11947197e-03, -4.09077890e-02,
        -2.29021758e-01,  1.38323888e-01, -1.69399288e-03,
        -1.14324324e-01],
       [ 7.81567022e-02,  3.76766846

In [5]:
previsoes = classificador.predict(previsores_teste)
previsoes = previsoes > 0.5

5/5 [==============================] - 0s 0s/step


In [6]:
accuracy_score(classe_teste, previsoes)

0.7482517482517482

In [7]:
confusion_matrix(classe_teste, previsoes)

array([[52,  4],
       [32, 55]], dtype=int64)

In [8]:
resultado = classificador.evaluate(previsores_teste, classe_teste)
resultado

5/5 [==============================] - 0s 2ms/step - loss: 0.8714 - binary_accuracy: 0.7483


[0.871420681476593, 0.748251736164093]

Adicionando Dropout e aplicando cross validation.

In [46]:
def criarRede1(): 
    k.clear_session()
    classificador = Sequential([
               tf.keras.layers.Dense(units=16, activation = 'relu', kernel_initializer = 'random_uniform', input_dim=30),
               tf.keras.layers.Dense(units=16, activation = 'relu', kernel_initializer = 'random_uniform'),
               tf.keras.layers.Dropout(0.2),
               tf.keras.layers.Dense(units=1, activation = 'sigmoid')])
    otimizador = tf.keras.optimizers.Adam(learning_rate = 0.001, weight_decay = 0.0001, clipvalue = 0.5)
    classificador.compile(optimizer = otimizador, loss = 'binary_crossentropy', metrics = ['binary_accuracy'])
    return classificador

In [47]:
classificador = KerasClassifier(model = criarRede1,
                                epochs = 100,
                                batch_size = 10)

resultados = cross_val_score(estimator = classificador,
                             X = previsores, y = classe,
                             cv = 2, scoring = 'accuracy')

print(resultados.mean())
print(resultados.std())

Epoch 1/100
29/29 [==============================] - 1s 2ms/step - loss: 1.0629 - binary_accuracy: 0.6268
Epoch 2/100
29/29 [==============================] - 0s 1ms/step - loss: 0.6624 - binary_accuracy: 0.6585
Epoch 3/100
29/29 [==============================] - 0s 2ms/step - loss: 0.5361 - binary_accuracy: 0.7289
Epoch 4/100
29/29 [==============================] - 0s 2ms/step - loss: 0.5375 - binary_accuracy: 0.7430
Epoch 5/100
29/29 [==============================] - 0s 2ms/step - loss: 0.5576 - binary_accuracy: 0.7218
Epoch 6/100
29/29 [==============================] - 0s 2ms/step - loss: 0.5831 - binary_accuracy: 0.7430
Epoch 7/100
29/29 [==============================] - 0s 1ms/step - loss: 0.5081 - binary_accuracy: 0.7077
Epoch 8/100
29/29 [==============================] - 0s 2ms/step - loss: 0.4861 - binary_accuracy: 0.8063
Epoch 9/100
29/29 [==============================] - 0s 2ms/step - loss: 0.4304 - binary_accuracy: 0.8063
Epoch 10/100
29/29 [==========================

Aplicando Tuning.

In [41]:
def criarRede2(optimizer='adam', loss='binary_crossentropy'):
    k.clear_session()
    classificador = Sequential([
               tf.keras.layers.Dense(units=16, activation = 'relu', kernel_initializer = 'random_uniform', input_dim=30),
               tf.keras.layers.Dropout(0.2),
               tf.keras.layers.Dense(units=16, activation = 'relu', kernel_initializer = 'random_uniform'),
               tf.keras.layers.Dropout(0.2),
               tf.keras.layers.Dense(units=1, activation = 'sigmoid')])
    classificador.compile(optimizer = optimizer, loss = loss, metrics = ['binary_accuracy'])
    return classificador

In [ ]:
classificador = KerasClassifier(build_fn = criarRede2)

parametros = {
    'batch_size': [10, 30],
    'epochs': [50, 100],
    'optimizer': ['adam', 'sgd'],
    'loss': ['binary_crossentropy', 'hinge']
}

grid_search = GridSearchCV(estimator=classificador, param_grid=parametros, scoring='accuracy', cv=5)
grid_search = grid_search.fit(previsores, classe)

In [43]:
print(grid_search.best_params_)
print(grid_search.best_score_)

{'batch_size': 30, 'epochs': 100, 'loss': 'binary_crossentropy', 'optimizer': 'adam'}
0.903400093153237


Salvando em formato JSON.

In [50]:
classificador = Sequential()

classificador.add(Dense(units = 16, activation = 'relu', 
                        kernel_initializer = 'random_uniform', input_dim = 30))
classificador.add(Dropout(0.2))
classificador.add(Dense(units = 16, activation = 'relu', 
                        kernel_initializer = 'random_uniform'))
classificador.add(Dropout(0.2))
classificador.add(Dense(units = 1, activation = 'sigmoid'))

classificador.compile(optimizer = 'adam', loss = 'binary_crossentropy',
                      metrics = ['binary_accuracy'])

classificador.fit(previsores, classe, batch_size = 30, epochs = 100)

classificador_json = classificador.to_json()
with open('classificador_breast.json', 'w') as json_file:
    json_file.write(classificador_json)
classificador.save_weights('classificador_breast.h5')

Epoch 1/100
19/19 [==============================] - 1s 2ms/step - loss: 1.7735 - binary_accuracy: 0.5571
Epoch 2/100
19/19 [==============================] - 0s 2ms/step - loss: 0.7837 - binary_accuracy: 0.5923
Epoch 3/100
19/19 [==============================] - 0s 2ms/step - loss: 0.5997 - binary_accuracy: 0.6292
Epoch 4/100
19/19 [==============================] - 0s 2ms/step - loss: 0.5785 - binary_accuracy: 0.6467
Epoch 5/100
19/19 [==============================] - 0s 2ms/step - loss: 0.5354 - binary_accuracy: 0.6555
Epoch 6/100
19/19 [==============================] - 0s 2ms/step - loss: 0.5244 - binary_accuracy: 0.6784
Epoch 7/100
19/19 [==============================] - 0s 2ms/step - loss: 0.5190 - binary_accuracy: 0.6801
Epoch 8/100
19/19 [==============================] - 0s 2ms/step - loss: 0.4992 - binary_accuracy: 0.7047
Epoch 9/100
19/19 [==============================] - 0s 4ms/step - loss: 0.4970 - binary_accuracy: 0.6854
Epoch 10/100
19/19 [==========================

Carregando rede neural.

In [51]:
arquivo = open('classificador_breast.json', 'r')
estrutura_rede = arquivo.read()
arquivo.close()

classificador = model_from_json(estrutura_rede)
classificador.load_weights('classificador_breast.h5')

novo = np.array([[15.80, 8.34, 118, 900, 0.10, 0.26, 0.08, 0.134, 0.178,
                  0.20, 0.05, 1098, 0.87, 4500, 145.2, 0.005, 0.04, 0.05, 0.015,
                  0.03, 0.007, 23.15, 16.64, 178.5, 2018, 0.14, 0.185,
                  0.84, 158, 0.363]])

In [52]:
previsores = pd.read_csv('entradas_breast.csv')
classe = pd.read_csv('saidas_breast.csv')
classificador.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
classificador.evaluate(previsores, classe)

18/18 [==============================] - 0s 2ms/step - loss: 0.1213 - binary_accuracy: 0.9455


[0.12133578211069107, 0.945518434047699]